In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# mcg 단위를 mg로 변환하는 함수
def convert_mcg_to_mg(value):
    # '영양 성분 정보' 문자열 제거
    value = re.sub(r'영양 성분 정보.*?%$', '', value)
    # 숫자와 단위를 추출하기 위한 정규표현식
    match = re.search(r'([\d,]+\.?\d*)([a-z A-Z %]+)', value)
    if match:
        numeric_value = float(match.group(1).replace(',', ''))  # 쉼표(,) 제거 후 숫자 값 추출
        unit = match.group(2)  # 단위 추출
        # mcg 단위를 mg로 변환
        if unit.lower() == 'mcg':
            mg_value = round(numeric_value / 1000, 3)
            return f"{mg_value}mg"
        # IU 단위를 mg로 변환
        elif unit.lower() == 'iu':
            mg_value = round(numeric_value / 0.67)
            return f"{mg_value}mg"
        # mg 단위 그대로 반환
        elif unit.lower() == 'mg' or unit.lower() == '%':
            return match.group(0)
    return value

# iHerb 사이트에서 제품 정보를 크롤링하는 함수
def crawl_iherb(query_txt):
    # iHerb 검색 URL 생성
    url = f"https://kr.iherb.com/search?kw={query_txt}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 제품 목록을 선택
    items = soup.select('.product-listing .product-inner-container')
    data = []

    for item in items:
        try:
            # 각 제품 정보 추출
            name = item.select_one(".product-title a").text.strip()  # 제품 제목
            link = item.select_one(".product-title a")["href"]  # 제품 상세 링크
            price = item.select_one(".price").text.strip()  # 제품 가격
            product_per_price = item.select_one(".price-per-unit").text.strip() if item.select_one(".price-per-unit") else "N/A"  # 제품 개별 가격

            # 제품 상세 페이지 요청
            product_response = requests.get(f"https://kr.iherb.com{link}")
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            # 상세 페이지에서 추가 정보 추출
            product_manufacture = product_soup.select_one('span[itemprop="brand"] > a').text.strip()  # 제조사
            product_usage = product_soup.select_one('.prodOverviewDetail').text.strip() if product_soup.select_one('.prodOverviewDetail') else "N/A"  # 사용 방법
            product_precautions = product_soup.select_one('.prodOverviewDetail:nth-of-type(2)').text.strip() if product_soup.select_one('.prodOverviewDetail:nth-of-type(2)') else "N/A"  # 주의 사항
            product_ingredient_information = product_soup.select_one('.supplement-facts-container').text.strip() if product_soup.select_one('.supplement-facts-container') else "N/A"  # 영양 성분 정보

            # 영양 성분 정보 단위 변환
            converted_ingredient_info = convert_mcg_to_mg(product_ingredient_information)

            # 리뷰 추출
            reviews = product_soup.select('.review-text-content')
            reviews_str = "\n".join([review.text.strip() for review in reviews])  # 리뷰를 줄바꿈으로 연결

            # 추출한 정보를 리스트에 추가
            data.append({
                'Product_Title': name,
                'Product_Manufacture': product_manufacture,
                'Product_Price': price,
                'Product_Per_Price': product_per_price,
                'Product_Usage': product_usage,
                'Product_Precautions': product_precautions,
                'Product_Ingredient_Information': converted_ingredient_info,
                'Review': reviews_str
            })
        except Exception as e:
            print(f"Error processing item: {e}")
            continue

    return data

# 키워드 입력받기
query_txt = input('크롤링할 키워드는 무엇입니까?')

# 크롤링 수행
data = crawl_iherb(query_txt)

# pandas DataFrame으로 변환
df = pd.DataFrame(data)

# DataFrame 표시
df

""
